In [11]:

import pandas as pd
import numpy as np
from datetime import datetime



1. Transformacion de datos

In [12]:
def transformar_datos(df):

    df_transformed = df.copy()
    

    # Calcular métricas de ventas
    df_transformed['Revenue_per_Unit'] = df_transformed['Total Amount'] / df_transformed['Quantity']
    df_transformed['Sales_Normalized'] = (df_transformed['Total Amount'] - df_transformed['Total Amount'].min()) / \
                                       (df_transformed['Total Amount'].max() - df_transformed['Total Amount'].min())
    
    # Clasificar ventas
    conditions = [
        (df_transformed['Total Amount'] <= 100),
        (df_transformed['Total Amount'] <= 500),
        (df_transformed['Total Amount'] > 500)
    ]
    choices = ['Baja', 'Media', 'Alta']
    df_transformed['Sales_Category'] = np.select(conditions, choices, default='No Clasificado')
    
    return df_transformed

2. Agregacion y agrupacion

In [13]:
def analizar_grupos(df):
    
    # Análisis por categoría de producto
    product_analysis = df.groupby('Product Category').agg({
        'Total Amount': ['count', 'sum', 'mean', 'std'],
        'Quantity': ['sum', 'mean'],
        'Transaction ID': 'count'
    }).round(2)
    

    # Análisis demográfico
    demographic_analysis = df.groupby(['Gender', 'Product Category']).agg({
        'Total Amount': ['sum', 'mean', 'count'],
        'Age': ['mean', 'min', 'max']
    }).round(2)
    
    return {
        'product_analysis': product_analysis,
        'demographic_analysis': demographic_analysis
    }

3. Análisis Personalizado con apply

In [14]:
def analisis_personalizado(df):
    """
    Ejercicio de análisis personalizado usando apply que incluye:
    - Funciones personalizadas
    - Cálculos específicos por grupo
    - Transformaciones avanzadas
    """
    # Función para calcular el percentil de venta dentro de su categoría
    def calcular_percentil(x):
        return x.rank(pct=True)
    
    # Función para determinar si una venta está por encima de la media de su categoría
    def comparar_con_media(x):
        return 'Por encima' if x > 0 else 'Por debajo'
    
    # Crear DataFrame para análisis personalizado
    df_custom = df.copy()
    
    # Calcular percentil de ventas por categoría
    df_custom['Sales_Percentile'] = df_custom.groupby('Product Category')['Total Amount'].transform(calcular_percentil)
    
    # Calcular desviación respecto a la media de la categoría
    df_custom['Category_Mean_Deviation'] = df_custom.groupby('Product Category')['Total Amount'].transform(
        lambda x: x - x.mean()
    )
    
    # Clasificar ventas respecto a la media
    df_custom['Performance_vs_Mean'] = df_custom['Category_Mean_Deviation'].apply(comparar_con_media)
    
    # Calcular métricas avanzadas
    df_custom['Price_per_Unit_Rank'] = df_custom.groupby('Product Category')['Price per Unit'].transform(
        lambda x: x.rank(method='dense')
    )
    
    return df_custom

Ejecucion de metodos

In [15]:
df = pd.read_csv('../data/retail_sales_dataset.csv')
df_transformada = transformar_datos(df)
print("\nResultados de transformación:")
print(df_transformada.head())

resultados_grupos = analizar_grupos(df)
print("\nResultados de agrupación:")
print(resultados_grupos['product_analysis'])

df_custom = analisis_personalizado(df)
print("\nResultados de análisis personalizado:")
print(df_custom.head())


Resultados de transformación:
   Transaction ID        Date Customer ID  Gender  Age Product Category  \
0               1  2023-11-24     CUST001    Male   34           Beauty   
1               2  2023-02-27     CUST002  Female   26         Clothing   
2               3  2023-01-13     CUST003    Male   50      Electronics   
3               4  2023-05-21     CUST004    Male   37         Clothing   
4               5  2023-05-06     CUST005    Male   30           Beauty   

   Quantity  Price per Unit  Total Amount  Revenue_per_Unit  Sales_Normalized  \
0         3              50           150              50.0          0.063291   
1         2             500          1000             500.0          0.493671   
2         1              30            30              30.0          0.002532   
3         1             500           500             500.0          0.240506   
4         2              50           100              50.0          0.037975   

  Sales_Category  
0          M